In [ ]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
    !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
    !git fetch --all
    !git reset --hard origin/master

In [ ]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
#     !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
#     !git pull

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime
importlib.reload(tradestats)
from tradestats import *

In [ ]:
prices = loaddata_1D('AUDUSD_Candlestick_1_D_BID_03.08.2003-24.10.2021.csv')
# prices = loaddata_1D('EURUSD_Candlestick_1_D_BID_03.08.2003-24.10.2021.csv')

In [ ]:
len(prices)

In [ ]:
dictList = []
colFeat = []
keyList = []

maKey  = [3,5,7,30,35,40,45,60,80,100,120,140,160,200]
atrKey = [20]
# srsKey = [0]
rsiKey = [15]

maDict   = ma2(prices, maKey,3,5,7)
atrDict  = atr(prices, atrKey)
# srsDict  = srs(prices, srsKey)
rsiDict = rsi(prices,rsiKey)

dictList.append(maDict.df); colFeat.append('ma'); keyList.append(maKey) 
dictList.append(atrDict.df); colFeat.append('atr'); keyList.append(atrKey) 
# dictList.append(srsDict.df); colFeat.append('srs'); keyList.append(srsKey) 
dictList.append(rsiDict.df); colFeat.append('rsi'); keyList.append(rsiKey) 

tt = [1]
sl = [0.6,0.65,0.7,0.75,0.8,0.85,0.9]
tp = [0.7,0.75,0.8,0.85,0.9]
tsl = [0,0.3,0.5,0.7]
atrperiod = 'atr20atr_prev'
savetrades = True

In [ ]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','month','day','date','weekday','open','high','low','close','volume']] = prices[['id','year','month','day','date','weekday','open','high','low','close','volume']]   

In [ ]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [ ]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [ ]:
masterFrame = masterFrame.dropna()

In [ ]:
starttime = datetime.now()
# alltrades = preparetrades_brut_tsl(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2015,2021)
# alltrades = preparetrades_brut_tp(masterFrame,[1],[0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01],[0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01],2003,2021)
# alltrades = preparetrades_brut_tp(masterFrame,[-1,1],[0.5],[0.9],2020,2021,atrperiod)
# alltrades = preparetrades_brut_tp(masterFrame,[-1],[0.3,0.4,0.5,0.6,0.7,0.8,0.9],[0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5],2003,2021,atrperiod)
# alltrades = preparetrades_brut_tp(masterFrame,[1],[1,1.2,1.4,1.6,1.8,2,2.2],[1.2,1.4,1.6,1.8,2,2.2,2.4,2.6,2.8,3,3.2],2003,2021,atrperiod)
alltrades = preparetrades_brut_tsl(masterFrame,tt,sl,tp+[100],tsl,2003,2021,atrperiod)
# alltrades = preparetrades_brut_tsl(masterFrame,[1],sl,tp,tsl,2003,2021,atrperiod)
# alltrades = preparetrades_brut_tp(masterFrame,[-1],[0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01],[0.006,0.007,0.008,0.009,0.01,0.011,0.012,0.013,0.014,0.015],2003,2021,atrperiod)
# alltrades = preparetrades_brut_tp(masterFrame,[-1],[0.003,0.004,0.005,0.006,0.007,0.008,0.01,0.012,0.014],[0.003,0.004,0.005,0.006,0.007,0.008,0.01,0.012,0.014],2003,2021,atrperiod)
endtime = datetime.now()
print(str(endtime - starttime))

In [ ]:
trades = cleartrades_brut(alltrades.copy(),savetrades)

In [ ]:
def runstats43(alltrades,ma1,atrperiod,sl,tp,tsl,ma2 = 5,fxs=[]):
    ma2=str(ma2)
    stats = runstats_ma_v43(alltrades,'ma'+ma1+'SMAdiffseq_prev', 'ma'+ma1+'SMAdiffdiff_prev', 'ma'+ma1+'SMAvs'+ma2+'_prev', 'ma'+ma2+'SMAdiffseq_prev', 'ma'+ma2+'SMAdiffdiff_prev', 'rsi15rsidiffseq_prev', 'rsi15rsi_prev', atrperiod,sl,tp,tsl,'x1',fxs)

    return stats

def runstats_ma_v43(alltrades,a,b,sv,aa,bb,aaa,bbb,atr='atr140atr_prev',sl=[],tp=[],tsl=[],ff='',fxs=[]):
    conf   = {}
    params = {}
    
    params['tradetype'] = [2,[1]]
    params['sl'] =        [2,sl]
    params['tp'] =        [2,tp]
    params['tsl'] =       [2,tsl]
    params[atr]  =        [3,[-1000],[0.015]]
    params[a]    =        [0,[-5,-1,1,2,3,4],[-1,1,2,3,4,1000]]
#     params[b]    =        [0,[-1000,0,1000],[-1000,0,1000]]
    params[sv]    =       [0,[-1000,0,1000],[-1000,0,1000]]
#     params[aa]   =        [0,[-1000,0,1000],[-1000,0,1000]]
    params[bb]   =        [0,[-1000,0,1000],[-1000,0,1000]]
    params[aaa]    =      [0,[-5,-1,1,2,3,4],[-1,1,2,3,4,1000]]
    params[bbb]    =      [0,[0,30,40,50,60,70],[30,40,50,60,70,100]]

    conf['filename'] =    'ma_43_2003_2021_1_'+atr+'_'+sv+ff
    if (fxs!=[]):
        conf['fxs'] = fxs
    print(conf['filename'])
    stats = stathyperparams2(alltrades,params,conf)
    return stats

In [ ]:
runstats43(alltrades,'30',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'35',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'40',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'45',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'60',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'80',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'100',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'120',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'140',atrperiod,sl,tp,tsl,5)
runstats43(alltrades,'160',atrperiod,sl,tp,tsl,5)
    

# [3,5,7,30,35,40,45,60,80,100,120,140,160]

#     stats1 = runstats28(alltrades,'30',atrperiod,sl,tp,tsl)
#     stats1 = runstats28(alltrades,'40',atrperiod,sl,tp,tsl)
#     stats1 = runstats28(alltrades,'50',atrperiod,sl,tp,tsl)

#     stats1 = runstats28(alltrades,'60',atrperiod,sl,tp,tsl)
#     stats1 = runstats28(alltrades,'80',atrperiod,sl,tp,tsl)
#     stats1 = runstats28(alltrades,'100',atrperiod,sl,tp,tsl)

#     stats1 = runstats28(alltrades,'120',atrperiod,sl,tp,tsl)
#     stats1 = runstats28(alltrades,'140',atrperiod,sl,tp,tsl)
#     stats1 = runstats28(alltrades,'160',atrperiod,sl,tp,tsl)


In [ ]:
importlib.reload(tradestats)
from tradestats import *
# for i in [10,14,20,30,40]:
for i in [20]:
    atrperiod = 'atr'+str(i)+'atr_prev'
    fxs=runstats43(alltrades,'30',atrperiod,sl,tp,tsl,5,[325])

sub = calcandplot(alltrades,fxs)  